### Тема 11. Асинхронность в Python

Задание
Напишите загрузчик содержимого веб-страниц с использованием асинхронного подхода.

Список адресов веб-страниц можно задать в коде. 
Например:

sites = [
        "https://www.jython.org",
        "https://python.org/",
    ] * 80

1.  Используйте библиотеки aysncio и aiohttp

2. Корутины должны запускаться одновременно с помощью функции gather. Обратите внимание, что gather принимает на вход созданные корутины или объекты класса Task. 

3. Результатом посещения страницы должна быть печать количества символов на странице  
Например:  

> Read 10782 from https://www.jython.org

4. Для того, чтобы проверить, что все корутины работают, можно передавать номер корутины в качестве параметра функции и выводить на печать. 
Вывод может быть такой:

> Read 10782 from https://www.jython.org from coroutine 5

5. Реализуйте возможность задать количество корутин, которые будут запущены (с помощью input). Список адресов должен быть равномерно распределен по корутинам.
Например, если у нас в списке 10 сайтов, а задано 7 корутин, то разбиение должно быть таким: [2, 2, 2, 1, 1, 1, 1]

6. Обратить внимание на работу с клиентской сессией. Попробуйте варианты создания сессии внутри корутины, создания единой сессии на все корутины и передачу сессии в качестве аргумента. 

7. Посчитайте время работы программы с помощью библиотеки time. Сравните полученные временные результаты с приведенными примерами (в файле 11_asyncio.ipynb) с использованием потоков, процессов, ThreadPoolExecutor. 


In [1]:
import time
import asyncio
import aiohttp


In [2]:
# создаю массив массивов с url
sites = [ "https://www.jython.org", "https://python.org/", ] * 8
# количество корутин
coro_num = 2
arr = []
for i in range(0, coro_num):
    arr.append([])
for i in range(len(sites)):
    arr[i%coro_num].append(sites[i])
print(arr)

[['https://www.jython.org', 'https://www.jython.org', 'https://www.jython.org', 'https://www.jython.org', 'https://www.jython.org', 'https://www.jython.org', 'https://www.jython.org', 'https://www.jython.org'], ['https://python.org/', 'https://python.org/', 'https://python.org/', 'https://python.org/', 'https://python.org/', 'https://python.org/', 'https://python.org/', 'https://python.org/']]


In [3]:
# создаю функцию запроса из массива url
async def scrape_text_len(urls, coro_id):
    async with aiohttp.ClientSession() as session:
        for url in urls:
            async with session.get(url) as response:
                resp = await response.text()
                print(f"Read {len(resp)} from {url} from coroutine {coro_id}")

In [4]:
# создаю функцию создания массива тасок
# в каждой корутине своя сессия для запроса
def tasks_for_scrape(arr_with_urls):
    tasks = []
    for idx, urls in enumerate(arr_with_urls):
        task = asyncio.create_task(scrape_text_len(urls, idx+1))
        tasks.append(task)
    return tasks

In [7]:
# выполняю программу
async def main():
    start_time = time.time()
    tasks_arr = tasks_for_scrape(arr)
    await asyncio.gather(*tasks_arr)
    time_difference = time.time() - start_time
    print(f"общее время выполнения: %.2f секунд." % time_difference)
await main()

Read 10780 from https://www.jython.org from coroutine 1
Read 10780 from https://www.jython.org from coroutine 1
Read 50624 from https://python.org/ from coroutine 2
Read 10780 from https://www.jython.org from coroutine 1
Read 10780 from https://www.jython.org from coroutine 1
Read 10780 from https://www.jython.org from coroutine 1
Read 50624 from https://python.org/ from coroutine 2
Read 10780 from https://www.jython.org from coroutine 1
Read 10780 from https://www.jython.org from coroutine 1
Read 10780 from https://www.jython.org from coroutine 1
Read 50624 from https://python.org/ from coroutine 2
Read 50624 from https://python.org/ from coroutine 2
Read 50624 from https://python.org/ from coroutine 2
Read 50624 from https://python.org/ from coroutine 2
Read 50624 from https://python.org/ from coroutine 2
Read 50624 from https://python.org/ from coroutine 2
общее время выполнения: 3.39 seconds.
